# Set up a local Kubernetes cluster with `kind` and run a pod with a local Docker image
The purpose of this notebook is to show how a Docker image that has been built locally can be tested in a local Kubernetes cluster.

The only prerequisite for running this notebook is a running Docker daemon.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Create-Docker-image-with-a-simple-HTTP-server" data-toc-modified-id="Create-Docker-image-with-a-simple-HTTP-server-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Create Docker image with a simple HTTP server</a></span><ul class="toc-item"><li><span><a href="#Python-script" data-toc-modified-id="Python-script-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Python script</a></span></li><li><span><a href="#Build-Docker-image" data-toc-modified-id="Build-Docker-image-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Build Docker image</a></span></li></ul></li><li><span><a href="#Ensure-that-kubectl-and-kind-are-available" data-toc-modified-id="Ensure-that-kubectl-and-kind-are-available-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Ensure that <code>kubectl</code> and <code>kind</code> are available</a></span></li><li><span><a href="#Set-up-a-local-Kubernetes-cluster-with-kind" data-toc-modified-id="Set-up-a-local-Kubernetes-cluster-with-kind-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Set up a local Kubernetes cluster with <code>kind</code></a></span><ul class="toc-item"><li><span><a href="#Create-cluster" data-toc-modified-id="Create-cluster-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Create cluster</a></span></li><li><span><a href="#Verify-that-the-Kubernetes-cluster-is-running" data-toc-modified-id="Verify-that-the-Kubernetes-cluster-is-running-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Verify that the Kubernetes cluster is running</a></span></li></ul></li><li><span><a href="#Create-a-pod-in-the-cluster" data-toc-modified-id="Create-a-pod-in-the-cluster-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Create a pod in the cluster</a></span><ul class="toc-item"><li><span><a href="#Load-the-image-into-the-kind-cluster" data-toc-modified-id="Load-the-image-into-the-kind-cluster-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Load the image into the <code>kind</code> cluster</a></span></li><li><span><a href="#Start-a-pod-with-the-image" data-toc-modified-id="Start-a-pod-with-the-image-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Start a pod with the image</a></span></li><li><span><a href="#Wait-until-the-pod-is-ready" data-toc-modified-id="Wait-until-the-pod-is-ready-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Wait until the pod is ready</a></span></li><li><span><a href="#Forward-the-port-of-the-pod-to-a-local-port" data-toc-modified-id="Forward-the-port-of-the-pod-to-a-local-port-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Forward the port of the pod to a local port</a></span></li><li><span><a href="#Verify-that-the-HTTP-server-in-the-cluster-answers-Hello-world" data-toc-modified-id="Verify-that-the-HTTP-server-in-the-cluster-answers-Hello-world-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Verify that the HTTP server in the cluster answers <code>Hello world</code></a></span></li></ul></li><li><span><a href="#Delete-the-kind-cluster" data-toc-modified-id="Delete-the-kind-cluster-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Delete the <code>kind</code> cluster</a></span></li></ul></div>

## Create Docker image with a simple HTTP server

### Python script
Create a Python script (inspired by https://gist.github.com/davidbgk/b10113c3779b8388e96e6d0c44e03a74), which runs an HTTP server at port 8000 and responds `Hello world` to all GET requests.

In [1]:
export CONTAINER_PORT=8000

mkdir -p tmp

cat << EOF > tmp/server.py
import http.server
import socketserver
from http import HTTPStatus

class Handler(http.server.BaseHTTPRequestHandler):
    def do_GET(self):
        self.send_response(HTTPStatus.OK)
        self.end_headers()
        self.wfile.write(b'Hello world\n')

socketserver.TCPServer(('', $CONTAINER_PORT), Handler).serve_forever()
EOF

### Build Docker image

In [2]:
cat << EOF > tmp/Dockerfile
FROM python:3-slim
COPY server.py .
EXPOSE $CONTAINER_PORT
CMD python server.py
EOF

In [3]:
DOCKER_IMAGE_NAME=python-hello-world-server
docker build -t $DOCKER_IMAGE_NAME tmp

Sending build context to Docker daemon  4.096kB
Step 1/4 : FROM python:3-slim
 ---> ce689abb4f0d
Step 2/4 : COPY server.py .
 ---> Using cache
 ---> 449502d749bb
Step 3/4 : EXPOSE 8000
 ---> Using cache
 ---> ead62cb7db41
Step 4/4 : CMD python server.py
 ---> Using cache
 ---> 2b76a0b02e3b
Successfully built 2b76a0b02e3b
Successfully tagged python-hello-world-server:latest


## Ensure that `kubectl` and `kind` are available
Since this notebook has no prerequisites except Docker, we download the required tools to a subdirectory `download` of the current directory.

In [4]:
mkdir -p download
export DOWNLOAD_DIR=$PWD/download

if [[ ":$PATH:" != *":$DOWNLOAD_DIR:"* ]]; then
    export PATH=$DOWNLOAD_DIR:$PATH
fi

This helper function downloads a file from a given URL if it is not present yet in the download directory.

In [5]:
function download() {
    target=$1
    target_path="$DOWNLOAD_DIR/$target"
    url=$2
    
    if [ ! -f "$target_path" ]; then
        echo "Downloading $target from $url..."
        curl -sLo "$target_path" $url
        chmod u+x "$target_path"
    else
        echo "$target has already been downloaded."
    fi
}

In [6]:
download kubectl "https://storage.googleapis.com/kubernetes-release/release/$(curl -s https://storage.googleapis.com/kubernetes-release/release/stable.txt)/bin/linux/amd64/kubectl"

kubectl has already been downloaded.


In [7]:
download kind "https://kind.sigs.k8s.io/dl/v0.8.1/kind-$(uname)-amd64"

kind has already been downloaded.


## Set up a local Kubernetes cluster with `kind`

### Create cluster

In [8]:
export KUBECONFIG=tmp/kind-config
CLUSTER_NAME=kind-test-local-image
kind create cluster --name $CLUSTER_NAME

Creating cluster "kind-test-local-image" ...
 ✓ Ensuring node image (kindest/node:v1.18.2) 🖼7l
 ✓ Preparing nodes 📦 7l
 ✓ Writing configuration 📜7l
 ✓ Starting control-plane 🕹️7l
 ✓ Installing CNI 🔌7l
 ✓ Installing StorageClass 💾7l
Set kubectl context to "kind-kind-test-local-image"
You can now use your cluster with:

kubectl cluster-info --context kind-kind-test-local-image

Have a question, bug, or feature request? Let us know! https://kind.sigs.k8s.io/#community 🙂


### Verify that the Kubernetes cluster is running

In [9]:
kubectl cluster-info

Kubernetes control plane is running at https://127.0.0.1:43021
KubeDNS is running at https://127.0.0.1:43021/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


## Create a pod in the cluster

### Load the image into the `kind` cluster

In [10]:
kind load docker-image $DOCKER_IMAGE_NAME --name $CLUSTER_NAME

Image: "python-hello-world-server" with ID "sha256:2b76a0b02e3b7296dbf9b72171811002b941c6844b7bb66b299d4fcd5b7ff251" not yet present on node "kind-test-local-image-control-plane", loading...


### Start a pod with the image
Note that the option `--image-pull-policy IfNotPresent` is required. Without this option, Kubernetes will try to pull the image. This is not possible because the image exists only on the local machine.

In [11]:
kubectl run hello-world --image $DOCKER_IMAGE_NAME --image-pull-policy IfNotPresent

pod/hello-world created


### Wait until the pod is ready
This is important when running all cells in this notebook automatically. Even though the pod usually starts up pretty quickly, the next cells might be executed by Jupyter even faster.

In [12]:
kubectl wait --for=condition=ready pod/hello-world

pod/hello-world condition met


In [13]:
kubectl get pods

NAME          READY   STATUS    RESTARTS   AGE
hello-world   1/1     Running   0          3s


### Forward the port of the pod to a local port
Note:
* `kubectl port-forward` is run in the background.
* The first line of its output is captured according to https://stackoverflow.com/questions/20017805/bash-capture-output-of-command-run-in-background.

In [14]:
command='kubectl port-forward pod/hello-world :8000'
echo "Running: '$command'"
exec 3< <($command &)
read <&3 line
echo "Output:  '$line'"
HOST_PORT=$(echo $line | sed -r 's/^[^:]*:([0-9]+) .*$/\1/g')
echo "Port:    $HOST_PORT"

Running: 'kubectl port-forward pod/hello-world :8000'
Output:  'Forwarding from 127.0.0.1:36285 -> 8000'
Port:    36285


### Verify that the HTTP server in the cluster answers `Hello world`

In [15]:
curl 127.0.0.1:$HOST_PORT

Hello world


## Delete the `kind` cluster

In [16]:
kind delete cluster --name $CLUSTER_NAME

Deleting cluster "kind-test-local-image" ...
E0223 16:13:28.851323    4647 portforward.go:233] lost connection to pod
